## LSSTCam Sky Coverage
Scroll through on sky images to see coverage
Craig Lage - 23-Apr-25

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from lsst.summit.utils.plotting import plot
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath
from astropy.time import Time, TimeDelta
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.daf.butler import Butler


# Scroll through historical data to see AltAz values

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])

detectors = [['R00', 'SG0'], ['R00', 'SG1'], 
  ['R04', 'SG0'], ['R04', 'SG1'], 
  ['R40', 'SG0'], ['R40', 'SG1'],
  ['R44', 'SG0'], ['R44', 'SG1']]


In [ ]:
startDay = 20250406
endDay = 20250417

output = []
dayObs = startDay
while dayObs <= endDay:
    records = butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs)
    print(dayObs, records.count())
    if records.count() < 200:
        dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                       

        continue
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        exposureList.append(record.id)
    exposureList = sorted(exposureList)
    for expId in exposureList:
        for [raft, ccd] in detectors:
            try:
                seqNum = int(expId - dayObs * 1E5)
                filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
                rp = ResourcePath(filename)
                with rp.open(mode="rb") as f:
                    hdu_list = fits.open(f)
                [roiSeg, roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
                output.append([expId, raft, ccd, roiSeg])
                print(expId, raft, ccd, roiSeg)
            except:
                #print(f"{expId} failed!")
                continue
    print(f"Done with {dayObs}")
    print()
    dayObs = calcNextDay(dayObs)  # find the next dayObs 


In [ ]:
for out in output:
    print(out)